# Revista Evidenciação Contábil

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all('h3', {'class': 'title'})
    a_links = []
    for item in results:
        try:
            if 'Editorial' not in item.find('a').text:
                a_links.append(item.find('a')['href'])
        except:
            pass
    return a_links

In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [5]:
url_search_text = 'https://periodicos.ufpb.br/ojs2/index.php/recfin/search/index?query=__query__&searchJournal=69&authors=&dateFromMonth=1&dateFromDay=1&dateFromYear=2013&dateToMonth=12&dateToDay=31&dateToYear=2021&searchPage=__pg_num__#results'
links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[1]

link


'https://periodicos.ufpb.br/ojs2/index.php/recfin/article/view/43673'

In [6]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [8]:
def limpar_afiliacao(afiliacao):
    afiliacao = ' '.join(afiliacao.split())
    afiliacao = afiliacao.split('-')[0]
    afiliacao = afiliacao.split('(')[0]
    return afiliacao.strip()

In [9]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [10]:
captura_titulo(soup)

{'Título': 'Maiores ONGs do Setor Ambiental: Uma Análise da Diversificação das Receitas'}

In [11]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date[:10]}

In [12]:
captura_data(soup)

{'Data de Publicação': '2019-09-02'}

In [13]:
def captura_resumo(soup):
    try:
        abstract_section = soup.find('section', {'class': 'item abstract'})
        abstract = abstract_section.text
        abstract = abstract.replace('Resumo', '').strip()
        #abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [14]:
captura_resumo(soup)

{'Resumo': 'Objetivo: Identificar o nível de diversificação de receitas das maiores Organizações Não Governamentais (ONGs) ambientais do mundo.Fundamento: A credibilidade é um aspecto fundamental para o setor das organizações sem fins lucrativos, pois um equívoco administrativo pode comprometer os fluxos de caixa futuros ou até mesmo decretar o fim da organização (Conway et al., 2015). Nesse sentido, a composição e o nível de diversificação das receitas dessas organizações estão relacionados com a continuidade destas.Método: O demonstrativo contábil que evidencia as receitas auferidas das 34 ONGs do setor ambiental, relativos aos anos de 2017, 2016 e 2015, foi analisado. As ONGs estrangeiras comumente divulgam essa informação no Statement of Activities. Por meio da análise dos tipos de receita, foram elaboradas quatro categorias agregadoras, a saber: “Receita Externa”, “Receita Própria”, “Investimento e Juros” e “Outros”. A Receita Externa, por possuir características específicas, foi 

In [15]:
def captura_palavras_chave(soup):
    try:
        kw_section = soup.find('section', {'class': 'item keywords'})
        kw = kw_section.text.replace('Palavras-chave:', '')
        kw = ' '.join(kw.split())
        kw = kw.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    return {'Palavras-chave': kw}

In [16]:
captura_palavras_chave(soup)

{'Palavras-chave': 'ONGs. Statement of Activities. Evidenciação. Setor Ambiental'}

In [17]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [18]:
captura_autores(soup)

{'Qtd. de Autores': 3,
 'Autor 1': 'Fábio Minatto',
 'Autor 2': 'Ernesto Fernando Rodrigues Vicente',
 'Autor 3': 'José Alonso Borba',
 'Autor 4': '',
 'Autor 5': '',
 'Autor 6': ''}

In [19]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [' '.join(a.text.split()) for a in afiliacao]
        afiliacao = [limpar_afiliacao(a) for a in afiliacao]
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [20]:
captura_afiliacao(soup)

{'Afiliação 1': 'Universidade Federal de Santa Catarina',
 'Afiliação 2': 'Universidade Federal de Santa Catarina',
 'Afiliação 3': 'Universidade Federal de Santa Catarina',
 'Afiliação 4': '',
 'Afiliação 5': '',
 'Afiliação 6': ''}

In [21]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista Evidenciação Contábil'}
    url_search_text = 'https://periodicos.ufpb.br/ojs2/index.php/recfin/search/index?query=__query__&searchJournal=69&authors=&dateFromMonth=1&dateFromDay=1&dateFromYear=2013&dateToMonth=12&dateToDay=31&dateToYear=2021&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y-%m-%d')
    return df

## Pesquisa por artigos

In [22]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head()

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2019-05-24,Características Endógenas das Companhias Frent...,Objetivo: O presente artigo teve como objetivo...,Disclosure. Meio Ambiente. Nível. Teoria da Di...,2,Rafael Crisostomo Alves,Luiz Roberto Calado,,,,,Centro Universitário Alves Faria,Centro Universitário Alves Faria,,,,,https://periodicos.ufpb.br/ojs2/index.php/recf...,Revista Evidenciação Contábil
1,2019-09-02,Maiores ONGs do Setor Ambiental: Uma Análise d...,Objetivo: Identificar o nível de diversificaçã...,ONGs. Statement of Activities. Evidenciação. S...,3,Fábio Minatto,Ernesto Fernando Rodrigues Vicente,José Alonso Borba,,,,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,,,,https://periodicos.ufpb.br/ojs2/index.php/recf...,Revista Evidenciação Contábil
2,2016-08-24,Mudanças e Propostas Previstas no Relatório da...,Essa pesquisa tem por objetivo verificar se as...,Relatórios da Administração. Propostas Previst...,2,Edimar Mendes Andrade,Josilene da Silva Barbosa,,,,,Faculdade de Ciências Integradas do Pontal da ...,Faculdade de Ciências Integradas do Pontal da ...,,,,,https://periodicos.ufpb.br/ojs2/index.php/recf...,Revista Evidenciação Contábil
3,2014-11-28,Análise do Nível de Evidenciação de Informaçõe...,A preservação do meio ambiente tornou-se um te...,Contabilidade Ambiental. Evidenciação. Relatór...,4,Deisiane Alves Guimarães,Ilirio José Rech,Moisés Ferreira Cunha,Ivone Vieira Pereira,,,Universidade Federal de Uberlândia,Universidade Federal de Uberlândia,Universidade Federal de Goiás,Universidade de Rio Verde,,,https://periodicos.ufpb.br/ojs2/index.php/recf...,Revista Evidenciação Contábil
4,2019-09-02,Avaliação de Desempenho na Gestão de Recursos ...,"Objetivo: Esta pesquisa objetiva mapear, na li...",Gestão de Recursos Hídricos. Avaliação de Dese...,3,Sara Meurer,Sandra Rolim Ensslin,Hans Michael van Bellen,,,,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,,,,https://periodicos.ufpb.br/ojs2/index.php/recf...,Revista Evidenciação Contábil


In [23]:
df.to_csv('../data/RECFIN.csv', index=False)

In [24]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2019-05-24,Características Endógenas das Companhias Frent...,Objetivo: O presente artigo teve como objetivo...,Disclosure. Meio Ambiente. Nível. Teoria da Di...,2,Rafael Crisostomo Alves,Luiz Roberto Calado,,,,,Centro Universitário Alves Faria,Centro Universitário Alves Faria,,,,,https://periodicos.ufpb.br/ojs2/index.php/recf...,Revista Evidenciação Contábil
1,2019-09-02,Maiores ONGs do Setor Ambiental: Uma Análise d...,Objetivo: Identificar o nível de diversificaçã...,ONGs. Statement of Activities. Evidenciação. S...,3,Fábio Minatto,Ernesto Fernando Rodrigues Vicente,José Alonso Borba,,,,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,,,,https://periodicos.ufpb.br/ojs2/index.php/recf...,Revista Evidenciação Contábil
2,2016-08-24,Mudanças e Propostas Previstas no Relatório da...,Essa pesquisa tem por objetivo verificar se as...,Relatórios da Administração. Propostas Previst...,2,Edimar Mendes Andrade,Josilene da Silva Barbosa,,,,,Faculdade de Ciências Integradas do Pontal da ...,Faculdade de Ciências Integradas do Pontal da ...,,,,,https://periodicos.ufpb.br/ojs2/index.php/recf...,Revista Evidenciação Contábil
3,2014-11-28,Análise do Nível de Evidenciação de Informaçõe...,A preservação do meio ambiente tornou-se um te...,Contabilidade Ambiental. Evidenciação. Relatór...,4,Deisiane Alves Guimarães,Ilirio José Rech,Moisés Ferreira Cunha,Ivone Vieira Pereira,,,Universidade Federal de Uberlândia,Universidade Federal de Uberlândia,Universidade Federal de Goiás,Universidade de Rio Verde,,,https://periodicos.ufpb.br/ojs2/index.php/recf...,Revista Evidenciação Contábil
4,2019-09-02,Avaliação de Desempenho na Gestão de Recursos ...,"Objetivo: Esta pesquisa objetiva mapear, na li...",Gestão de Recursos Hídricos. Avaliação de Dese...,3,Sara Meurer,Sandra Rolim Ensslin,Hans Michael van Bellen,,,,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,,,,https://periodicos.ufpb.br/ojs2/index.php/recf...,Revista Evidenciação Contábil
5,2018-04-28,Evidenciação da Informação Contábil: uma Retro...,Objetivo: Este estudo objetiva analisar as car...,Evidenciação da informação contábil. Trabalhos...,2,Iago Franca Lopes,Ilse Maria Beuren,,,,,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,,,,,https://periodicos.ufpb.br/ojs2/index.php/recf...,Revista Evidenciação Contábil
6,2019-01-24,Variáveis Contingenciais e Sistemas de Control...,Objetivo: Identificar a percepção dos líderes ...,Variáveis Contingenciais. Sistemas de Controle...,3,Lara Fabiana Dallabona,Leonardo Tomasoni Nardelli,Ana Rita Venzon Fernandes,,,,Universidade do Estado de Santa Catarina,Universidade do Estado de Santa Catarina,Universidade do Estado de Santa Catarina,,,,https://periodicos.ufpb.br/ojs2/index.php/recf...,Revista Evidenciação Contábil
7,2014-11-28,Logística Reversa: Análise Bibliométrica de Ar...,O presente trabalho objetiva identificar a evo...,Logística Reversa. Bibliometria. Meio Ambiente...,3,Ferdinando David Valandro,Marcia Zanievicz Silva,Júlio César Silva,,,,Instituto Nacional de Pós,Universidade Regional de Blumenau,Universidade Regional de Blumenau,,,,https://periodicos.ufpb.br/ojs2/index.php/recf...,Revista Evidenciação Contábil
8,2017-08-30,Relato integrado e o formato da informação fin...,RESUMOObjetivo: Descrever forma e conteúdo de ...,Relato integrado. Criação de valor. Informação...,4,Nadson Jaime Ferreira Alves,José Roberto Kassai,Edimilson Costa Lucas,Humberto Medrado Gomes Ferreira,,,Professor da Universidade Federal do Pará Dout...,Universidade de São Paulo,Professor do programa de pós graduação em Admi...,Doutorando em Administração da Universidade Mu...,,,https://periodicos.ufpb.br/ojs2/in